<a href="https://colab.research.google.com/github/danielmancovidales/Aplicaciones-en-salud/blob/main/prepro_usuarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [206]:
# Librerias generales
import pandas as pd
import numpy as np
import re
# Librerias y visualización
import seaborn as sns
import matplotlib.pyplot as plt

In [207]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [208]:
# Leer base de datos
df_usuarios = pd.read_csv('/content/drive/MyDrive/analitica3/Salud/RETO_df_usuarios.csv', encoding='latin1', sep=',', low_memory= False)
df_usuarios

,MES,YEAR,NRODOC,SEXO,FECHA NACIMIENTO,EDAD,DEPARTAMENTO,MUNICIPIO,BARRIO,FECHA INICIO AL PGP,FECHA PRIMERA CLASE FUNCIONAL,PRIMERA CLASE FUNCIONAL,FECHA ÃLTIMA CLASE FUNCIONAL,ÃLTIMA CLASE FUNCIONAL,CICLO_VITAL,QUINQUENIO
0,ABRIL,2017,306221,F,1938-09-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABRIL,2017,306222,F,1974-07-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABRIL,2017,306223,F,1930-08-24 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABRIL,2017,306224,F,1979-12-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABRIL,2017,306225,M,1943-06-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183906,JUNIO,2022,307482,F,1941-03-26 00:00:00,81.0,ANTIOQUIA,MEDELLIN,NaN,2017-04-01 00:00:00,2017-08-23 00:00:00,CLASE FUNCIONAL 2B,2022-04-22 00:00:00,CLASE FUNCIONAL 2A,Vejez,80-84
183907,JUNIO,2022,313801,F,1999-04-13 00:00:00,23.0,ANTIOQUIA,NaN,NaN,2021-09-02 00:00:00,2021-10-26 00:00:00,CLASE FUNCIONAL 1,2022-01-27 00:00:00,CLASE FUNCIONAL 1,Juventud,20-24
183908,JUNIO,2022,307738,F,1943-09-11 00:00:00,78.0,ANTIOQUIA,MEDELLIN,NaN,2017-04-01 00:00:00,2017-06-14 00:00:00,CLASE FUNCIONAL 2B,2022-01-06 00:00:00,CLASE FUNCIONAL 2A,Vejez,75-79
183909,JUNIO,2022,311656,M,1940-05-26 00:00:00,82.0,ANTIOQUIA,MEDELLIN,NaN,2018-01-01 00:00:00,2018-02-21 00:00:00,CLASE FUNCIONAL 2A,2022-03-17 00:00:00,CLASE FUNCIONAL 2B,Vejez,80-84


**Cambio de nombres y eliminación de duplicados**

In [209]:
# Se cambian los nombres de mayusculas a minisculas
df_usuarios.columns = df_usuarios.columns.str.lower()

In [210]:
#se verifican la variables existentes
df_usuarios.columns

Index(['mes', 'year', 'nrodoc', 'sexo', 'fecha nacimiento', 'edad',
       'departamento', 'municipio', 'barrio', 'fecha inicio al pgp',
       'fecha primera clase funcional', 'primera clase funcional',
       'fecha ãltima clase funcional', 'ãltima clase funcional',
       'ciclo_vital', 'quinquenio'],
      dtype='object')

In [211]:
# Función para quitar los espacios de los nombres y reemplazar por '_'
def limpiar_nombre_columna(nombre):
    nombre_limpio = nombre.replace(" ", "_")  # Reemplazar espacios por guiones bajos
    return nombre_limpio

# Aplicar la función a los nombres de las columnas
df_usuarios.columns = df_usuarios.columns.map(limpiar_nombre_columna)

In [212]:
# Se corrigen los nombres que tienen tildes
df_usuarios = df_usuarios.rename(columns={'fecha_ãltima_clase_funcional': 'fecha_ultima_clase_funcional'})
df_usuarios = df_usuarios.rename(columns={'ãltima_clase_funcional': 'ultima_clase_funcional'})

In [213]:
#Identificación de filas duplicadas
df_usuarios.duplicated().sum()

5012

In [214]:
# Se eliminan las 5012 filas duplicadas
df_usuarios = df_usuarios.drop_duplicates()

In [215]:
#Identificación de pacientes diferentes

print("El número de registros es:", df_usuarios['nrodoc'].count())
print("El número de pacientes distintos es:", len(df_usuarios['nrodoc'].unique()))

El número de registros es: 178899
El número de pacientes distintos es: 8068


**Identificación de nulos y eliminación de variables poco relevantes**

In [216]:
#Conteo de nulos de cada variable
df_usuarios.isnull().sum()

mes                                  0
year                                 0
nrodoc                               0
sexo                                 7
fecha_nacimiento                 72076
edad                             29144
departamento                     96615
municipio                        97502
barrio                           63555
fecha_inicio_al_pgp              53241
fecha_primera_clase_funcional    90938
primera_clase_funcional          89258
fecha_ultima_clase_funcional     90914
ultima_clase_funcional           50790
ciclo_vital                      29144
quinquenio                       29144
dtype: int64

Se pueden borrar algunas columnas que resultan irrelevantes y/o redundantes en el caso de estudio como "fecha_nacimiento", "fecha_primera_clase_funcional", "fecha_ultima_clase_funcional" ya que hay una variable de edad y la fecha en que se realizaron las clases funcionales no aportan datos importantes.
Finalmente se elimina la variable "fecha_inicio_al_pgp" debido a que la antiguedad no aporta datos importantes para determinar la permanencia del usuario al programa.

In [217]:
#Se pueden borrar algunas columnas que resultan irrelevantes y se corrobora que se hizo bien el borrado
df_usuarios = df_usuarios.drop(['fecha_nacimiento', 'fecha_primera_clase_funcional', 'fecha_ultima_clase_funcional', 'fecha_inicio_al_pgp'], axis=1)
df_usuarios.head()

,mes,year,nrodoc,sexo,edad,departamento,municipio,barrio,primera_clase_funcional,ultima_clase_funcional,ciclo_vital,quinquenio
0,ABRIL,2017,306221,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABRIL,2017,306222,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABRIL,2017,306223,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABRIL,2017,306224,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABRIL,2017,306225,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [218]:
#Se consulta si hay diferentes departamentos
df_usuarios["departamento"].value_counts()

departamento
ANTIOQUIA    82284
Name: count, dtype: int64

In [219]:
#se consulta el contenido de algunas variables por medio de una variable con la terminación "view"
#ya que sólo se va a utilizar en esta línea de código
df_usuarios_view = df_usuarios[["departamento", "municipio", "barrio"]]
df_usuarios_view.value_counts().head(30)

departamento  municipio             barrio              
ANTIOQUIA     MEDELLIN              MANRIQUE                2606
                                    ARANJUEZ                1876
                                    CASTILLA                1655
                                    BUENOS AIRES            1361
                                    CAMPO VALDES            1274
              BELLO                 NIQUIA                  1083
                                    BELLO                   1074
              MEDELLIN              SAN JAVIER              1061
                                    12 DE OCTUBRE           1045
                                    BELEN                   1015
                                    ROBLEDO                  951
                                    VILLA HERMOSA            862
              ITAGUI                ITAGUI                   844
              SAN ANTONIO DE PRADO  SAN ANTONIO DE PRADO     818
              MEDELLIN           

El único departamento en el DF es Antioquia junto a sus respectivos municipios y barrios, por lo que se procede a borrar estas 3 variables ya que la geografía no es una variable importante dentro del caso de estudio.

In [220]:
#Se borran las variables y se corrobora que se hizo el cambio correctamente
df_usuarios = df_usuarios.drop(["departamento", "municipio", "barrio"], axis=1)
df_usuarios.head(10)

,mes,year,nrodoc,sexo,edad,primera_clase_funcional,ultima_clase_funcional,ciclo_vital,quinquenio
0,ABRIL,2017,306221,F,NaN,NaN,NaN,NaN,NaN
1,ABRIL,2017,306222,F,NaN,NaN,NaN,NaN,NaN
2,ABRIL,2017,306223,F,NaN,NaN,NaN,NaN,NaN
3,ABRIL,2017,306224,F,NaN,NaN,NaN,NaN,NaN
4,ABRIL,2017,306225,M,NaN,NaN,NaN,NaN,NaN
5,ABRIL,2017,306226,M,NaN,NaN,NaN,NaN,NaN
6,ABRIL,2017,306227,F,NaN,NaN,NaN,NaN,NaN
7,ABRIL,2017,306228,M,NaN,NaN,NaN,NaN,NaN
8,ABRIL,2017,306229,F,NaN,NaN,NaN,NaN,NaN
9,ABRIL,2017,306230,F,NaN,NaN,NaN,NaN,NaN


**Identificación de registros mal escritos o atípicos**

In [221]:
#Se consulta si los meses están bien escritos
df_usuarios["mes"].value_counts()

mes
JUNIO         18679
MAYO          18649
ENERO         14308
OCTUBRE       14294
DICIEMBRE     14293
SEPTIEMBRE    14182
FEBRERO       14158
JULIO         14102
AGOSTO        14077
NOVIEMBRE     14063
MARZO         14057
ABRIL         14037
Name: count, dtype: int64

In [222]:
#Se cambian a minúsculas y se corrobora el cambio
df_usuarios['mes'] = df_usuarios['mes'].str.lower()
df_usuarios['mes'].value_counts()

mes
junio         18679
mayo          18649
enero         14308
octubre       14294
diciembre     14293
septiembre    14182
febrero       14158
julio         14102
agosto        14077
noviembre     14063
marzo         14057
abril         14037
Name: count, dtype: int64

In [223]:
#Se consulta si los años están bien escritos
df_usuarios['year'].value_counts()

year
2021    55229
2018    52915
2017    43563
2022    27192
Name: count, dtype: int64

Se puede notar que no hay una continuidad en la serie de tiempo de los años ya que faltan los años 2019 y 2020. Puede ser por errores de digitación o porque efectivamente falta una parte del DF. En el porterior análisis exploratorio se decidirá si se sigue trabajando con estos datos o se van a depurar.

In [224]:
#Se consulta si los sexos están bien escritos
df_usuarios['sexo'].value_counts()

sexo
F                                       112220
M                                        66658
CARRERA 51 B 1 33                            1
CL 48  95 55                                 1
CR 49 46 14                                  1
CRR 47 N  41 32                              1
CALLE 80 A 71 A 18                           1
CL 94  74B  16                               1
CALLE 77E   8830                             1
KR 55A 47 16  BR SANTA ANA                   1
KR 92 NRO 46 97                              1
CL 55 80 54                                  1
CALLE 38 SUR N  29 A 54 PISO 2               1
CR 75A   88A   27 BRR ROBLEDO KENEDY         1
CL  40 C  SUR   29 C  31LA FLORIDA           1
CL 31 NRO 44 159                             1
Name: count, dtype: int64

In [225]:
#Al tener errores en la toma de los datos se procede a sólo tomar en cuenta los registros "F" y "M"

#Se crea una lista de valores a mantener
genero = ['F', 'M']

#Se iltran las filas donde 'SEXO' esté en la lista de valores a mantener y se corrobora el cambio
df_usuarios = df_usuarios[df_usuarios['sexo'].isin(genero)]
df_usuarios['sexo'].value_counts()

sexo
F    112220
M     66658
Name: count, dtype: int64

In [226]:
#Se consultan las clases funcionales
df_usuarios['primera_clase_funcional'].value_counts()

primera_clase_funcional
CLASE FUNCIONAL 2A    16537
CLASE FUNCIONAL 2B    16394
CLASE FUNCIONAL 4      8180
Clase funcional 2B     7398
Clase funcional 2A     6830
clase funcional 2b     5883
Clase funcional 4      5794
clase funcional 2a     5727
CLASE FUNCIONAL 3      5529
clase funcional 4      3738
CLASE FUNCIONAL 1      1927
clase funcional 3      1593
Sin Clasificar         1393
Clase funcional 3      1351
Clase funcional 1       533
clase funcional 1       518
Sin clasificar          316
Name: count, dtype: int64

Se puede notar que los registros están escritos de manera diferente para clases funcionales iguales por lo que se procede a cambiar todos los registros a minúsculas y así tener uniformidad.

In [227]:
#Se cambian a minúsculas y se corrobora el cambio
df_usuarios['primera_clase_funcional'] = df_usuarios['primera_clase_funcional'].str.lower()
df_usuarios['primera_clase_funcional'].value_counts()

primera_clase_funcional
clase funcional 2b    29675
clase funcional 2a    29094
clase funcional 4     17712
clase funcional 3      8473
clase funcional 1      2978
sin clasificar         1709
Name: count, dtype: int64

In [228]:
#Se consultan las clases funcionales
df_usuarios['ultima_clase_funcional'].value_counts()

ultima_clase_funcional
CLASE FUNCIONAL 2A    28462
CLASE FUNCIONAL 2B    23850
Clase funcional 2A    21800
CLASE FUNCIONAL 4     14993
Clase funcional 4     14283
Clase funcional 2B    13103
CLASE FUNCIONAL 3      5556
Clase funcional 3      1855
CLASE FUNCIONAL 1      1667
Sin Clasificar         1393
Clase funcional 1       671
Sin clasificar          284
SIN CLASIFICAR          192
Name: count, dtype: int64

In [229]:
#Se cambian a minúsculas y se corrobora el cambio
df_usuarios['ultima_clase_funcional'] = df_usuarios['ultima_clase_funcional'].str.lower()
df_usuarios['ultima_clase_funcional'].value_counts()

ultima_clase_funcional
clase funcional 2a    50262
clase funcional 2b    36953
clase funcional 4     29276
clase funcional 3      7411
clase funcional 1      2338
sin clasificar         1869
Name: count, dtype: int64

In [230]:
#Se consultan las categorías de ciclo vital
df_usuarios['ciclo_vital'].value_counts()

ciclo_vital
Vejez       130595
Adultez      17542
Juventud      1618
Name: count, dtype: int64

In [231]:
#Se cambian a minúsculas y se corrobora el cambio
df_usuarios['ciclo_vital'] = df_usuarios['ciclo_vital'].str.lower()
df_usuarios['ciclo_vital'].value_counts()

ciclo_vital
vejez       130595
adultez      17542
juventud      1618
Name: count, dtype: int64

In [232]:
#Se consultan las categorías de quinquenio
df_usuarios['quinquenio'].value_counts()

quinquenio
75-79     24956
70-74     23335
80-84     22711
65-69     18600
85-89     16554
60-64     12720
>90       11719
55-59      7855
50-54      4432
45-49      2107
25-29      1102
40-44      1095
35-39       939
30-34       927
20-24       678
15-19        25
Name: count, dtype: int64

**Eliminación y depuración de nulos**

In [233]:
df_usuarios.isnull().sum()

mes                            0
year                           0
nrodoc                         0
sexo                           0
edad                       29123
primera_clase_funcional    89237
ultima_clase_funcional     50769
ciclo_vital                29123
quinquenio                 29123
dtype: int64

In [234]:
#Total de filas que debería tener cada variable dado que la variable mes no tiene nulos
df_usuarios["mes"].count()

178878

In [235]:
print("El porcentaje de nulos de edad es:", (df_usuarios["edad"].isnull().sum()/178878)*100,"%")
print("El porcentaje de nulos de primera_clase_funcional es:", (df_usuarios["primera_clase_funcional"].isnull().sum()/178878)*100,"%")
print("El porcentaje de nulos de ultima_clase_funcional es:", (df_usuarios["ultima_clase_funcional"].isnull().sum()/178878)*100,"%")
print("El porcentaje de nulos de ciclo_vital es:", (df_usuarios["ciclo_vital"].isnull().sum()/178878)*100,"%")
print("El porcentaje de nulos de quinquenio es:", (df_usuarios["quinquenio"].isnull().sum()/178878)*100,"%")

El porcentaje de nulos de edad es: 16.28092890126231 %
El porcentaje de nulos de primera_clase_funcional es: 49.88707387157728 %
El porcentaje de nulos de ultima_clase_funcional es: 28.381913930164693 %
El porcentaje de nulos de ciclo_vital es: 16.28092890126231 %
El porcentaje de nulos de quinquenio es: 16.28092890126231 %


-El porcentaje de nulos en edad, ciclo_vital y quinquenio es igual, lo que significa que son 3 varibles que explican lo mismo por lo que se decide dejar ciclo vital y así tener menos categorías y optimizar la realización del posterior exploratorio.


-Los nulos en las clases funcionales se van a rellenar con la categoría "sin clasificar".

-Los nulos en el ciclo vital se van a llenar con el dato más frecuente.

In [249]:
df_usuarios = df_usuarios.drop(["edad", "quinquenio"], axis=1)

In [250]:
# Se decide reemplazar los nulos con la categoria 'sin clasificar'
df_usuarios['primera_clase_funcional'].fillna('sin clasificar', inplace=True)

In [251]:
# Se decide reemplazar los nulos en la categoria 'sin clasificar'
df_usuarios['ultima_clase_funcional'].fillna('sin clasificar', inplace=True)

In [252]:
#Consultar valor más común
df_usuarios['ciclo_vital'].value_counts()

ciclo_vital
vejez       159718
adultez      17542
juventud      1618
Name: count, dtype: int64

In [253]:
#Reemplazar por valor más común
df_usuarios['ciclo_vital'].fillna('vejez', inplace=True)

In [254]:
#Se corroboran todos los cambios
df_usuarios.isnull().sum()

mes                        0
year                       0
nrodoc                     0
sexo                       0
primera_clase_funcional    0
ultima_clase_funcional     0
ciclo_vital                0
dtype: int64

In [256]:
df_usuarios

,mes,year,nrodoc,sexo,primera_clase_funcional,ultima_clase_funcional,ciclo_vital
0,abril,2017,306221,F,sin clasificar,sin clasificar,vejez
1,abril,2017,306222,F,sin clasificar,sin clasificar,vejez
2,abril,2017,306223,F,sin clasificar,sin clasificar,vejez
3,abril,2017,306224,F,sin clasificar,sin clasificar,vejez
4,abril,2017,306225,M,sin clasificar,sin clasificar,vejez
...,...,...,...,...,...,...,...
183906,junio,2022,307482,F,clase funcional 2b,clase funcional 2a,vejez
183907,junio,2022,313801,F,clase funcional 1,clase funcional 1,juventud
183908,junio,2022,307738,F,clase funcional 2b,clase funcional 2a,vejez
183909,junio,2022,311656,M,clase funcional 2a,clase funcional 2b,vejez


**El DF se redujo de 183.912 a 178.878 filas y de 16 a 7 variables**



In [258]:
#convertimos la base a un csv
df_usuarios.to_csv('/content/drive/MyDrive/analitica3/Salud/df_usuarios.csv', index=True)